### Import & Settings

In [1]:
import warnings, importlib
warnings.filterwarnings('ignore')

import pandas as pd
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)

In [2]:
import sys
from pathlib import Path
project_dir = Path().resolve().parent
sys.path.append(str(project_dir))

In [3]:
from data.crypto_data_loader import load_multi_symbol_data, DataHandler

from model.feature_generator import FeatureGenerator, FeatureProcessor
from model.fit_pred import split_data, clean_xy, fit_predict_last_line
from model.signal_generator import SignalGenerator

handler = DataHandler()

In [4]:
# data symbols
target = 'ETHUSDT'
load_symbols = ['BTCUSDT', 'ETHUSDT','SOLUSDT','DOGEUSDT']

INTERVAL = '1h'
START_DELTA = timedelta(days=60)
# DB_PATH = 'data/crypto_data.db'
MODEL_NAME = 'rf-reg'
STRATEGY_NAME = 'zscore_atr_v1'
target_col = 'target_1h'

target_cols = ['target_1h','target_2h','target_4h','target_12h']

In [5]:
# model
from sklearn.ensemble import RandomForestRegressor
reg_rf = RandomForestRegressor(
    n_estimators=300,
    min_samples_leaf=5,
    max_features='sqrt',
    max_depth=15,
    random_state=42,
    n_jobs=-1)

train_model = reg_rf

In [6]:
# feature engineering params
feature_config = {
    "other_symbols": ['BTCUSDT', 'ETHUSDT','SOLUSDT','DOGEUSDT'],
    "lags": [1, 2, 4, 12, 24, 48, 72], # lags must contain rt_targets
    "rt_targets": [1, 2, 4, 12, 24],
    "vol_window": 24,
    "rsi_window": 12,
    "mfi_window": 12,
    "bb_window": 24,
    "tema_windows": [12,24,36],
    "adx_window": 12,
    "cmo_window": 12,
    "ulti_os_windows": {
            'period1':8,
            'period2':12,
            'period3':24
        },
    "patterns": {
            'CDLENGULFING': '吞没形态',
            'CDLHAMMER': '锤子线',
            'CDLHANGINGMAN': '吊人线',
            'CDLDOJI': '十字星',
            'CDLDRAGONFLYDOJI': '蜻蜓十字星',
            'CDLGRAVESTONEDOJI': '墓碑十字星',
            'CDLMORNINGSTAR': '晨星',
            'CDLEVENINGSTAR': '暮星',
            'CDLSHOOTINGSTAR': '流星',
            'CDLMARUBOZU': '光头光脚'
        }
}

# cols to scale and drop
feature_process_config = {
    'metrics_to_scale': ["high", "low", "close"],
    'cols_to_drop': ["open","high","low","close","volume","dollar_vol","hour",
                "atr","macdhist",'hour_bin_label','cmo','tema_12','tema_24','tema_36']
}

### 阶段 2：接通pipeline

In [75]:
import pipeline
import data.db_utils as dbu

importlib.reload(pipeline)
importlib.reload(dbu)

<module 'data.db_utils' from 'D:\\PersonalFiles\\Development\\Git Projects\\data-analytics-portfolio\\trading-signal\\data\\db_utils.py'>

#### 初始化

In [8]:
DB_PATH = '../data/crypto_data.db'
conn = dbu.get_connection(DB_PATH)

In [66]:
dbu.init_db(conn) # 初始化几张表

In [ ]:
# 拉取并 upsert 最近 72 小时的K线到 kline 表
pipeline.fetch_and_store_backfill_no_lag(conn, handler, symbols = load_symbols, backfill_hours=72)

# （可选）粗检 kline 表是否有数据
pd.read_sql_query("SELECT symbol, COUNT(*) AS n FROM kline GROUP BY symbol", conn)

Fetching price data backfill: [2025-08-24 14:59:00, 2025-08-27 14:59:00] for 4 symbols ...
Upserting 288 rows into kline (backfill 72h, no lag, allow provisional bars)...


,symbol,n
0,BTCUSDT,72
1,DOGEUSDT,72
2,ETHUSDT,72
3,SOLUSDT,72


#### 历史回填

In [9]:
df_processed, price_all = pipeline.prepare_feature_data_from_start(handler=handler,
                                                        symbols=load_symbols,
                                                        target=target,
                                                        start_str='2025-08-15',
                                                        buffer_windows=300,
                                                        train_windows=24*7*4)

In [10]:
from model.fit_pred import split_data, clean_xy

X, y = split_data(df_processed)
X_clean, y_clean = clean_xy(X, y[target_col])

In [12]:
from model.timeseries_cv import MultipleTimeSeriesCV
tscv = MultipleTimeSeriesCV(train_length=24*7*4, test_length=1, lookahead=1, date_idx='datetime')

In [13]:
pred_df = pipeline.train_and_predict(df_processed, cv=tscv)

In [76]:
# 只一步
start_str = (datetime.utcnow().replace(minute=0, second=0, microsecond=0) - timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S")
print(start_str)

pred_hist_df = pipeline.backfill_predictions_from(handler=handler,
                                                  start_str=start_str, 
                                                  symbols=load_symbols,
                                                  target=target, 
                                                  buffer_windows=300,
                                                  train_windows=24*7*4)
print("pred_hist_df.shape:", None if pred_hist_df is None else pred_hist_df.shape)

2025-08-22 01:00:00
pred_hist_df.shape: (167, 4)


#### 历史预测存入predictions表

In [77]:
dbu.upsert_df(pred_hist_df.reset_index(), 'predictions', conn)

#### 获取predictions表里的最大datetime
填充缺失的预测

In [52]:
max_dt = dbu.get_last_timestamp(conn, table='predictions')
print(max_dt)

2025-08-28 19:00:00


In [53]:
pred_fill_df = pipeline.backfill_predictions_from(handler=handler,
                                                  start_str=max_dt, 
                                                  symbols=load_symbols,
                                                  target=target, 
                                                  buffer_windows=300,
                                                  train_windows=24*7*4)

In [54]:
dbu.upsert_df(pred_fill_df.reset_index(), 'predictions', conn)

#### 只预测最后一条

In [32]:
start_str = (datetime.utcnow().replace(minute=0, second=0, microsecond=0) - timedelta(hours=2)).strftime("%Y-%m-%d %H:%M:%S")
print(start_str)

df_processed, price_all = pipeline.prepare_feature_data_from_start(handler=handler,
                                                                   symbols=load_symbols,
                                                                   target=target,
                                                                   start_str=start_str,
                                                                   buffer_windows=300,
                                                                   train_windows=24*7*4)

2025-08-28 11:00:00


In [21]:
X, y = split_data(df_processed)
X_clean, y_clean = clean_xy(X, y[target_col])

In [22]:
out = fit_predict_last_line(model=train_model, X=X_clean, y=y_clean, train_length=24*7*4)
print(out)

{'timestamp': Timestamp('2025-08-27 23:00:00'), 'y_pred': -0.0010631664009917643}


In [73]:
# 只一步
latest = pipeline.predict_latest(handler=handler,
                              symbols=load_symbols,
                             target=target,
                             buffer_windows=300,
                             train_windows=24*7*4)
print(latest)

    symbol             datetime  predicted model_name
0  ETHUSDT  2025-08-29 00:00:00   0.000083     rf-reg


#### 最后一条存入predictions表

In [74]:
dbu.upsert_df(latest, 'predictions', conn)

#### 从predition表生成信号

In [78]:
# 取近期历史预测
until_dt = (
    pd.Timestamp.now(tz=timezone.utc)   # 带UTC时区的 pandas Timestamp
      .floor('H')                       # 对齐到整点
      .tz_convert(None)                 # 去时区，变“naive UTC”
)
hist_pred = dbu.fetch_predictions_history(conn, symbol=target, until_dt=until_dt, lookback_hours=24*5)

In [79]:
# 获取近期价格数据
start_dt = (hist_pred.index.min() - pd.Timedelta(hours=24)).strftime('%Y-%m-%d %H:%M:%S')
price_hist = load_multi_symbol_data(handler, symbols=load_symbols, start_str=start_dt)

In [80]:
# 生成信号
from pipeline import signal_config

pred_df = hist_pred.copy()
pred_df.index.name = 'datetime'
# pred_df = pred_df.reset_index()
pred_df['symbol'] = target

df_signal = (
    SignalGenerator(config=signal_config)
    .load_data(pred_df=pred_df, price_df=price_hist)
    .merge_pred_price(target_symbol=target)
    .zscore_normalize()
    .compute_raw_signal()
    .apply_atr_volatility_filter()
    .compute_positions()
    .compute_reversals()
    .apply_min_signal_spacing(min_space=2)
)

In [81]:
df_signal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 120 entries, 2025-08-24 01:00:00 to 2025-08-29 00:00:00
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   predicted        120 non-null    float64
 1   symbol           120 non-null    object 
 2   open             120 non-null    float64
 3   high             120 non-null    float64
 4   low              120 non-null    float64
 5   close            120 non-null    float64
 6   volume           120 non-null    float64
 7   zscore           97 non-null     float64
 8   raw_signal       120 non-null    int64  
 9   vol_filter       120 non-null    bool   
 10  filtered_signal  120 non-null    int64  
 11  position         120 non-null    int64  
 12  signal_reversal  120 non-null    int64  
 13  final_signal     120 non-null    int64  
dtypes: bool(1), float64(7), int64(5), object(1)
memory usage: 17.3+ KB


In [82]:
df_signal['model_name'] = MODEL_NAME
df_signal['strategy_name'] = STRATEGY_NAME
df_signal['actuals'] = df_signal['close'].shift(-1) / df_signal['close'] - 1

#### 信号插入signals库表

In [83]:
dbu.upsert_df(df_signal.reset_index(),'signals', conn)

### 阶段 1：支持“最后 1 小时预测

In [12]:
# Load data from api
model_start_date = (datetime.utcnow() - START_DELTA).strftime('%Y-%m-%d %H:%M:%S')
print(model_start_date)

df_price = load_multi_symbol_data(handler=handler, symbols=load_symbols, interval=INTERVAL, start_str=model_start_date)

2025-06-27 01:01:17


In [13]:
# Feature generation and processing
df_features = (
        FeatureGenerator(config=feature_config)
        .load_data(df=df_price)
        .select_symbols(target_symbol=target)
        .compute_volume_features()
        .compute_momentum_features()
        .compute_volatility_features()
        .compute_target_cols()
        .compute_time_dummies()
        .compute_tech_indicators()
        .compute_candle_patterns()
        .get_single_symbol_data(target_symbol=target)
    )

df_processed = (
        FeatureProcessor(config=feature_process_config)
        .load_data(df=df_features)
        .scale_metrics()
        .drop_cols()
        .df
    )

In [14]:
X, y = split_data(df_processed)
X_clean, y_clean = clean_xy(X, y[target_col])

In [15]:
X.tail(5)

,log_dollar_vol,log_dollar_vol_3d,return_1h,return_2h,return_4h,return_12h,return_24h,return_48h,return_72h,return_1h_lag1,...,DOGE_rsi,DOGE_bb_high,DOGE_bb_low,DOGE_adx,DOGE_plus_di,DOGE_minus_di,DOGE_bop,high_norm,low_norm,close_norm
datetime,,,,,,,,,,,,,,,,,,,,,
2025-08-25 21:00:00,21.225306,20.958404,-0.000228,-0.015986,-0.051714,-0.057042,-0.090911,-0.084339,-0.103640,-0.015762,...,21.323549,0.129622,-0.004492,45.828694,6.294771,41.899220,0.119741,1.008307,0.997195,0.999772
2025-08-25 22:00:00,20.899369,20.954401,0.009705,0.009476,-0.039935,-0.033550,-0.081398,-0.081665,-0.087231,-0.000228,...,26.608483,0.125200,0.003306,47.414647,8.329550,39.078427,0.542662,1.015333,1.000000,1.009705
2025-08-25 23:00:00,20.676289,20.957177,-0.003658,0.006012,-0.010071,-0.048662,-0.084563,-0.084360,-0.094428,0.009705,...,28.455001,0.113302,0.014964,48.682137,8.575027,37.310830,0.291005,1.000000,0.991253,0.996344
2025-08-26 00:00:00,21.417028,20.969658,-0.010164,-0.013785,-0.004440,-0.067826,-0.078572,-0.097359,-0.099281,-0.003658,...,26.640077,0.108928,0.023111,50.050813,7.889149,37.328964,-0.417417,1.004328,0.987143,0.989836
2025-08-26 01:00:00,18.496110,20.955197,-0.003220,-0.013351,-0.007419,-0.067410,-0.087556,-0.096153,-0.097006,-0.010164,...,25.976897,0.111676,0.021081,51.305432,7.737096,36.609497,-0.680000,1.000104,0.995891,0.996780


In [16]:
X_clean.tail()

,log_dollar_vol,log_dollar_vol_3d,return_1h,return_2h,return_4h,return_12h,return_24h,return_48h,return_72h,return_1h_lag1,...,DOGE_rsi,DOGE_bb_high,DOGE_bb_low,DOGE_adx,DOGE_plus_di,DOGE_minus_di,DOGE_bop,high_norm,low_norm,close_norm
datetime,,,,,,,,,,,,,,,,,,,,,
2025-08-25 20:00:00,22.259343,20.966946,-0.015762,-0.048947,-0.053263,-0.054079,-0.091027,-0.084442,-0.102949,-0.033716,...,20.062470,0.099557,0.018746,43.278815,6.766976,42.638902,-0.855867,1.005755,0.980171,0.984233
2025-08-25 21:00:00,21.225306,20.958404,-0.000228,-0.015986,-0.051714,-0.057042,-0.090911,-0.084339,-0.103640,-0.015762,...,21.323549,0.129622,-0.004492,45.828694,6.294771,41.899220,0.119741,1.008307,0.997195,0.999772
2025-08-25 22:00:00,20.899369,20.954401,0.009705,0.009476,-0.039935,-0.033550,-0.081398,-0.081665,-0.087231,-0.000228,...,26.608483,0.125200,0.003306,47.414647,8.329550,39.078427,0.542662,1.015333,1.000000,1.009705
2025-08-25 23:00:00,20.676289,20.957177,-0.003658,0.006012,-0.010071,-0.048662,-0.084563,-0.084360,-0.094428,0.009705,...,28.455001,0.113302,0.014964,48.682137,8.575027,37.310830,0.291005,1.000000,0.991253,0.996344
2025-08-26 00:00:00,21.417028,20.969658,-0.010164,-0.013785,-0.004440,-0.067826,-0.078572,-0.097359,-0.099281,-0.003658,...,26.640077,0.108928,0.023111,50.050813,7.889149,37.328964,-0.417417,1.004328,0.987143,0.989836


In [ ]:
def fit_predict_last(model, X, y, train_length=24*7*4):

    X_train = X.iloc[-1 - train_length:-1]
    y_train = y[-1 - train_length:-1]
    X_pred = X.iloc[-1:]
    # y_test = y[-1:]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_pred)

    ts = X_pred.index[-1]

    return {"timestamp": ts,
            "y_pred": float(y_pred[-1])}

In [ ]:
pred_output = fit_predict_last(
    model=train_model,
    X=X_clean,
    y=y_clean,
    train_length=24*7*4
)

print(pred_output)

In [34]:
train_length=24*7*4
X_train = X_clean.iloc[-1 - train_length:-1]

print(X_train.index.min(), X_train.index.max())

2025-07-29 00:00:00 2025-08-25 23:00:00


In [35]:
X_pred = X_clean.iloc[-1:]
print(X_pred.index.min(), X_pred.index.max())

2025-08-26 00:00:00 2025-08-26 00:00:00


In [36]:
y_train = y_clean[-1 - train_length:-1]
print(y_train.index.min(), y_train.index.max())

2025-07-29 00:00:00 2025-08-25 23:00:00


In [31]:
X_clean.index.min(), X_clean.index.max()

(Timestamp('2025-07-09 02:00:00'), Timestamp('2025-08-26 00:00:00'))

In [37]:
train_model.fit(X_train, y_train)
y_pred = train_model.predict(X_pred)

print(y_pred)

[-0.00108094]
